# 1. TF-IDF
1. 장점
    - 직관적인 해석이 가능함
2. 단점
    - 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생
    - 높은 차원을 가짐
    - 매우 sparse한 형태의 데이터임
3. Represent Items
    - Items을 벡터 형태로 표현. 도메인에 따라 다른 방법이 적용

# 2. Word2Vec
1. 통계기반의 방법 단점
    - 대규모 말뭉치를 다룰 때 메모리 상의 문제가 발생
        - 높은 차원을 가짐, 매우 sparse한 형태의 데이터임
        - 예) 100만개의 문서를 다루는 경우 : 
        -   100만개의 문서에 등장한 모든 단어를 추출해야 하고 이때 단어의 수는 1문서당 새로운 단어가 10개면,
        -   1000만개정도의 말뭉치가 형성됨. 즉, 100만 * 1000만의 매트릭스가 형성
    - 한번에 학습 데이터 전체를 진행함
        - 큰 작업을 처리하기 어려움
        - GPU와 같은 병렬처리를 기대하기 힘들다.
    - 학습을 통해서 개선하기가 어려움
2. 통계 기반 기법 vs. 추론 기반 기법
    - 통계 기반 기법 (배치학습) : 학습데이터 -> `TF-IDF`
    - 추론 기반 기법 (미니배치 학습) : 학습데이터 -> `신경망`
3. 추론 기반의 방법 (Word2Vec)
    - 추론 : 주변 단어(**맥락**)이 주어졌을 때 "?"에 무슨 단어(**중심단어**)가 들어가는지를 추측하는 작업
    - 1) CBOW 모델 : you ___ goodbye and I say hello.
    - 2) skip-gram 모델 : ___ say ___ and I say hello.
    - 추론 기반 기법 : 맥락을 입력하면 모델은 각 단어의 출현 확률을 출력한다.
        - 예시) `{ you, goodbye} -- 맥락` => `모델` => `확률분포`
        -   모델은 {you, say, goodbye, and, I ,hello, .}이라는 말뭉치 후보군중에서 하나를 예측함
        - 잘못 예측한 경우 => 학습을 통해서 점점 모델을 개선
4. 정의
    - Word2Vec은 단어간 유사도를 반영하여 단어를 벡터로 바꿔주는 임베딩 방법론이다. 
    -   원-핫벳터 형태의 sparse matrix가 가지는 단점을 해소하고자 저차원의 공간에 벡터로 매핑하는 것이 특징
    -   Word2Vec은 "비슷한 위치에 등장하는 단어들은 비슷한 의미를 가진다"라는 가정을 통해서 학습을 진행
    -   저차원에 학습된 단어의 의미를 분산하여 표현하기에 단어 간 유사도를 계산할 수 있음
    -   추천시스템에서는 단어를 구매 상품으로 바꿔서 구매한 패턴에 Word2Vec을 적용해서 비슷한 상품을 찾을 수 있다.

# 3-1. Word2Vec - CBOW
1. 알고리즘
    - CBOW는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법.
    -   반대로, Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법.
    -   예) you say goodbye and I say hello.
    -   CBOW 모델 : you ___ goodbye and I say hello.
    -   skip-gram 모델 : ___ say ___ and I say hello. => CBOW보다 성능이 좋고, 현업에서도 많이 사용하는 편.
    -       주변단어 : 주변에 있는 단어 (you, goodbye)
    -       중심단어 : 중간에 있는 단어 (say)
    -       윈도우 크기 : 주변을 몇 칸까지 볼 지에 대한 크기 (1)
2. 추론기반의 방법
    - 딥러닝이란 : input이 들어오면 은닉층과 레이어가 쌓이고 거기서 weight를 통해서 output을 출력한 다음
    -   그 output의 softmax를 취해서 이걸로 정답레이블과 비교, 가중치 업데이트

# 3-2. Word2Vec - CBOW : 작동 순서
1. One Hot Vector 형태의 입력값을 받는다.
    - You -> [Say] 예측     # 입력값 : You, 정답레이블 : Say
    - Goodbye -> [Say] 예측 # 입력값 : Goodbye, 정답레이블 : Say
    -   주변 단어 : 주변에 있는 단어 (you, goodbye)
    -   중심 단어 : 중간에 있는 단어 (say)
    -   윈도우 크기 : 주변을 몇 칸까지 볼 지에 대한 크기 (1),
    -       만일, 윈도우 크기가 2이면 you, goodbye, and를 통해서 [Say]를 예측
2. One Hot Vector 형태의 입력값을 Win (Weight input)과 곱
    - 초기의 Weight는 랜덤값으로 정해짐
    - hidden layer1과 hidden layer2를 평균낸 것이 은닉층
    -   W input 값은 단어 임베딩 값
3. Hidden state의 값을 W_out과 곱해서 Score를 추출한다.
4. Score에 Softmax를 취해서 각 단어가 나올 확률을 계산한다. (Softmax는 확률이 아닌 것을 확률로 만들어준다.)
    - 계산된 확률값을 정답 레이어와 비교
5. 정답과 Cross Entropy Loss를 계산
6. 5.에서 계산한 Loss를 가지고 Backpropagation 과정을 통해서 Weight를 업데이트

# 4. 코드 구현
- 참고 : kaggle.com/chocozzz/00-word2vec-1

In [1]:
# Word2Vec 알고리즘
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

ModuleNotFoundError: No module named 'gensim'

In [2]:
pip install --upgrade gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim   # Word2Vec의 알고리즘을 호출해주는 패키지

In [4]:
import warnings
warnings.filterwarnings(action="ignore")

In [7]:
movie = pd.read_csv('../kaggle/ratings.csv')
movie.head(2)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435


In [8]:
# 시간순으로 정렬해주는 함수 : SORT_VALUES 
movie = movie.sort_values(by="timestamp", ascending=True).reset_index(drop=True)
movie.head()

,userId,movieId,rating,timestamp
0,5274,105,2.0,825582938
1,5274,4,2.0,825582938
2,5274,3,2.0,825582938
3,5274,88,2.0,825583060
4,5274,74,3.0,825583152


In [9]:
# 영화의 Metadata를 불러와서 movieID에 맞는 TITLE을 구해준다.
meta = pd.read_csv("../kaggle/movies_metadata.csv")
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FALSE,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44'],
      dtype='object')

In [11]:
meta = meta.rename(columns={'id':'movieId'})    # 컬럼명 변경
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], how='left', on='movieId')

In [12]:
movie.head()

,userId,movieId,rating,timestamp,original_title
0,5274,105,2.0,825582938,Back to the Future
1,5274,4,2.0,825582938,NaN
2,5274,3,2.0,825582938,Varjoja paratiisissa
3,5274,88,2.0,825583060,Dirty Dancing
4,5274,74,3.0,825583152,War of the Worlds


In [13]:
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)

In [14]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()

,unique
userId,
1,"[Young and Innocent, Shuang ma lian huan, Cesa..."
2,"[La passion de Jeanne d'Arc, La belle et la bê..."
3,"[I Love You to Death, Once Were Warriors, Mons..."
4,"[Muxmäuschenstill, Batman & Robin, Hidalgo, 12..."
5,"[Star Trek III: The Search for Spock, The Sear..."


In [15]:
movie['original_title'].unique()

array(['Back to the Future', 'Varjoja paratiisissa', 'Dirty Dancing', ...,
       'The Squall', 'Alienate',
       'The Lost Medallion: The Adventures of Billy Stone'], dtype=object)

- Word2Vec 적용

In [16]:
# int 형식은 Word2Vec에서 학습이 안되기 때문에 String으로 변경해준다.
sentence = []
for user_sentence in agg['unique'].values:
    # map -> list 내의 모든 원소에 대해서 string으로 변환
    sentence.append(list(map(str, user_sentence)))

In [17]:
sentence

[['Young and Innocent',
  'Shuang ma lian huan',
  'Cesare deve morire',
  'Sleepless in Seattle',
  'Trois couleurs : Rouge',
  'Rocky Balboa',
  'License to Wed',
  'Confession of a Child of the Century',
  'Fools Rush In',
  'Shriek If You Know What I Did Last Friday the Thirteenth',
  'Les Quatre Cents Coups'],
 ["La passion de Jeanne d'Arc",
  'La belle et la bête',
  'Jarhead',
  'Donnie Darko',
  'The 39 Steps',
  'Almost Famous',
  'Four Rooms',
  'Mrs. Doubtfire',
  'Interview with the Vampire',
  '?�雄',
  "Pirates of the Caribbean: Dead Man's Chest",
  'Monty Python and the Holy Grail',
  'Hable con ella',
  'The Matrix Revolutions',
  'Parenthood',
  'A Nightmare on Elm Street',
  'Night on Earth'],
 ['I Love You to Death',
  'Once Were Warriors',
  'Monsoon Wedding',
  'Sleepless in Seattle',
  'Reservoir Dogs',
  'The Third Man',
  'My Super Ex-Girlfriend',
  'Fools Rush In'],
 ['Muxmäuschenstill',
  'Batman & Robin',
  'Hidalgo',
  '12 + 1',
  'Tuya de hun shi',
  'Reign 

- vector_size : weight의 크기
- window : 양 옆의 주변 단어를 몇 개까지 볼지
- workers : cpu 자원을 몇 개나 학습에 참여할지
- sg : 0 - CBOW / sg : 1 - SKIP-GRAM

In [21]:
# Wor2Vec의 학습을 진행
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, vector_size=20, window=5,
                            min_count=1, workers=4, epochs=200, sg=1)

In [22]:
# wv : 가중치 벡터 가지고 "Spider-Man 2"에 대해 계산 가능
# 메타데이터가 잘 구축이 안된 경우, 컨텐츠 기반 같은 경우 잘 되지 않는다.
embedding_model.wv.most_similar(positive=["Spider-Man 2"], topn=10)

[('Chill Factor', 0.8917562365531921),
 ('Solaris', 0.8608607649803162),
 ('Lammbock', 0.8411389589309692),
 ('Land of Plenty', 0.8381018042564392),
 ('Cop Land', 0.8338028192520142),
 ('Rumor Has It...', 0.8333099484443665),
 ('Infinity', 0.8299754858016968),
 ('Crime of Passion', 0.8280280232429504),
 ('Me, Myself & Irene', 0.8230196237564087),
 ('The Flintstones', 0.8195549845695496)]

# 5. 컨텐츠 기반 모델
1. 장점
    - 협업필터링은 다른 사용자들의 평점이 필요한 반면에, 자신의 평점만을 가지고 추천시스템을 만들 수 있음
    - item의 feature를 통해서 추천을 하기에 추천이 된 이유를 설명하기 용이함
    - 사용자가 평점을 매기지 않은 새로운 item이 들어올 경우에도 추천이 가능함
2. 단점
    - item의 feature을 추출해야 하고 이를 통해서 추천하기 때문에 제대로 feature을 추출하지 못할 경우 정확도가 낮다.
    -   그렇기에 Domain Knowledge가 분석시에 필요할 수도 있다.
    - 기존의 item과 유사한 item 위주로만 추천하기에 새로운 장르의 item을 추천하기 어려움
    - 새로운 사용자에 대해서 충분한 평점이 쌓이기 전까지는 추천하기 힘들다.